#  Imports


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Activation, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.applications import Xception, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.metrics import f1_score


import seaborn as sns
import matplotlib.pyplot as plt
import os

# Function Definitions

In [3]:
def train_val_generators(train_dir, val_dir):
    train_gen = ImageDataGenerator(rescale=1/255., horizontal_flip=True, vertical_flip=True, zoom_range=0.2)
    train_generator = train_gen.flow_from_directory(directory=train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

    val_gen = ImageDataGenerator(rescale=1/255.)
    val_generator = val_gen.flow_from_directory(directory=val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

    return train_generator, val_generator


def train_model(model, train_generator, val_generator, epochs=15):
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

    history = model.fit(train_generator, validation_data=val_generator, epochs=epochs, callbacks=[callback])

    return history


In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def train_val_generators_shear(train_dir, val_dir):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )

    val_datagen = ImageDataGenerator(rescale=1./255.)

    train_generator = train_datagen.flow_from_directory(
        directory=train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    val_generator = val_datagen.flow_from_directory(
        directory=val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    return train_generator, val_generator


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

import seaborn as sns
import matplotlib.pyplot as plt

def plot_training_history(history):
    # Plotear el historial de entrenamiento
    eps = range(len(history.history["loss"]))
    plt.figure(figsize=(10, 6))
    plt.plot(eps, history.history["loss"])
    plt.plot(eps, history.history["val_loss"])
    plt.legend(["loss", "val_loss"])
    plt.title('Training History - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()

    # Plotear la precisión
    plt.figure(figsize=(10, 6))
    plt.plot(eps, history.history["accuracy"])
    plt.plot(eps, history.history["val_accuracy"])
    plt.legend(["accuracy", "val_accuracy"])
    plt.title('Training History - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()

def check_precision(model, test_generator):
    # Asignar e imprimir etiquetas
    test_labels, pred_labels = assign_labels(test_generator, model, print_info=False)

    # Imprimir la matriz de confusión y la precisión
    cm = confusion_matrix(test_labels, pred_labels)
    accuracy = accuracy_score(test_labels, pred_labels)

    print("Matriz de Confusión:")
    print(cm)
    print("Precisión:", accuracy)
    return cm





In [5]:
def plot_confusion_matrix(cm, classes):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Matriz de Confusión')
    plt.xlabel('Etiquetas Predichas')
    plt.ylabel('Etiquetas Verdaderas')
    plt.show()

# Configuracion inicial

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:

# Directorio base después de montar el drive
base_path = '/content/drive/My Drive/DataSet'

# Listar archivos en el directorio base
files = os.listdir(base_path)

path_root = '/content/drive/My Drive/DataSet'

valid_dir = os.path.join(path_root, 'val')
train_dir = os.path.join(path_root, 'train')
test_dir = os.path.join(path_root, 'test')

In [8]:
bc_types = [file for file in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, file))]
print("Types: ", bc_types)

# to encode labels
bc_dict = {'InSitu': 3, 'Benign': 2, 'Normal': 1, 'Invasive': 4}
print("Encode: ", bc_dict)

# to decode labels
dict_bc = {1: "Normal", 2: "Benign", 3: "InSitu", 4: "Invasive"}
print("Decode: ", dict_bc)

classes = [dict_bc[i] for i in sorted(dict_bc.keys())]


Types:  ['InSitu', 'Normal', 'Invasive', 'Benign']
Encode:  {'InSitu': 3, 'Benign': 2, 'Normal': 1, 'Invasive': 4}
Decode:  {1: 'Normal', 2: 'Benign', 3: 'InSitu', 4: 'Invasive'}


In [9]:
# Set IMG_SIZE and BATCH_SIZE
IMG_SIZE = (512, 512)
BATCH_SIZE = 8
NUM_CLASES = 4


In [10]:
import imghdr  # Importar el módulo imghdr para verificar el tipo de imagen

# Calcular el rango de píxeles en lugar de asumir que es 255
pixel_min = float('inf')  # Inicializar con un valor grande
pixel_max = float('-inf')  # Inicializar con un valor pequeño

# Iterar sobre las imágenes en el conjunto de entrenamiento para encontrar el rango real
for root, dirs, files in os.walk(train_dir):
    for file in files:
        img_path = os.path.join(root, file)

        # Verificar si el archivo es una imagen
        if imghdr.what(img_path) is not None:
            img = plt.imread(img_path)
            img_min = np.min(img)
            img_max = np.max(img)
            pixel_min = min(pixel_min, img_min)
            pixel_max = max(pixel_max, img_max)

In [11]:
pixel_min

0

In [12]:
train_generator, val_generator = train_val_generators(train_dir, valid_dir)   # create image generators to feed images to the model


Found 236 images belonging to 4 classes.
Found 81 images belonging to 4 classes.


In [32]:
train_generator_2, val_generator_2 = train_val_generators_shear(train_dir, valid_dir)   # create image generators to feed images to the model


Found 236 images belonging to 4 classes.
Found 81 images belonging to 4 classes.


# Create models

In [13]:
#Para poder incluir el valor f1 en el entrenamiento del modelo creamos las siguientes
#funciones para añadirlo como una función siguiendo la referencia del sigueinte enlace
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



## ENTRENAMIENTO DE MODELOS

In [14]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import Xception, EfficientNetB0, InceptionV3, VGG16, VGG19, ResNet50
from tensorflow.keras.optimizers import Adam
import tensorflow as tf


In [15]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models

def build_model_with_base(base_model, num_classes, img_size1, img_size2, learning_rate, freeze_layers=None, additional_layers=None):
    inputs = layers.Input(shape=(img_size1, img_size2, 3))

    # Configurar el modelo base
    base_model = base_model(include_top=False, input_tensor=inputs, weights="imagenet")

    # Congelar todas las capas si freeze_layers es None
    if freeze_layers == 'all':
        for layer in base_model.layers:
            layer.trainable = False
    elif isinstance(freeze_layers, int):
        # Congelar las primeras 'freeze_layers' capas
        for layer in base_model.layers[:freeze_layers]:
            layer.trainable = False
    else:
        raise ValueError("El argumento freeze_layers debe ser 'all' o un entero.")

    # Construir el modelo con el modelo base
    x = base_model.output

    # Agregar capas adicionales si se proporcionan
    if additional_layers:
        for layer in additional_layers:
            x = layer(x)

    # Agregar capas de convolución adicionales
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2, 2)(x)

    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2, 2)(x)

    # Agregar capas finales
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2, 2)(x)

    # Agregar GlobalAveragePooling2D
    x = GlobalAveragePooling2D()(x)

    x = layers.Dropout(0.3)(x)
    x = Dense(64, activation="relu")(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    # Compilar el modelo
    model = models.Model(inputs, outputs, name=f"CustomModel_{base_model.name}")
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['acc', f1_m, precision_m, recall_m])

    return model


In [16]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models

def build_model_with_base_v2(base_model, num_classes, img_size1, img_size2, learning_rate, freeze_layers=None, additional_layers=None):
    inputs = layers.Input(shape=(img_size1, img_size2, 3))
    base_model = base_model(include_top=False, input_tensor=inputs, weights="imagenet")

    # Congelar todas las capas si freeze_layers es None
    if freeze_layers == 'all':
        for layer in base_model.layers:
            layer.trainable = False
    elif isinstance(freeze_layers, int):
        # Congelar las primeras 'freeze_layers' capas
        for layer in base_model.layers[:freeze_layers]:
            layer.trainable = False
    else:
        raise ValueError("El argumento freeze_layers debe ser 'all' o un entero.")

    # Construir el modelo con el modelo base

    x = base_model.output

    # Reducir el número de capas de convolución
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2, 2)(x)

    # Agregar una capa de convolución adicional
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2, 2)(x)

    # Agregar GlobalAveragePooling2D
    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    # Compilar el modelo
    model = models.Model(inputs, outputs, name=f"CustomModel_{base_model.name}")
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['acc', f1_m, precision_m, recall_m])

    return model


In [17]:
xception_model_preprocessed = build_model_with_base(Xception, NUM_CLASES, IMG_SIZE[0], IMG_SIZE[1],0.00005,0)


83683744/83683744 [==============================] - 0s 0us/step


In [18]:
xception_model_preprocessed_2 = build_model_with_base_v2(Xception, NUM_CLASES, IMG_SIZE[0], IMG_SIZE[1],0.00005,0)


# Entrenamiento de modelos

In [19]:
# Specify the directory to save the models
save_dir = '/content/drive/My Drive/SavedModels'

# Ensure the directory exists, create it if necessary
os.makedirs(save_dir, exist_ok=True)

In [20]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)   # limitation to stop somewhere early


In [21]:
print("\n[INFO] Ready to train. Training is starting!\n")
BATCH_SIZE = 8



[INFO] Ready to train. Training is starting!



In [22]:
# Model training for Xception
%%time

hist_xception_preprocessed = xception_model_preprocessed.fit(train_generator, validation_data=val_generator, epochs=50)
# Guardar modelos en formato H5
xception_model_preprocessed.save(os.path.join(save_dir, 'xception_model_preprocessed.h5'))


Epoch 1/50
30/30 [==============================] - 108s 2s/step - loss: 1.9011 - acc: 0.2542 - f1_m: 0.2207 - precision_m: 0.2559 - recall_m: 0.1958 - val_loss: 1.3860 - val_acc: 0.2716 - val_f1_m: 0.0202 - val_precision_m: 0.0909 - val_recall_m: 0.0114
Epoch 2/50
30/30 [==============================] - 21s 676ms/step - loss: 1.6048 - acc: 0.2924 - f1_m: 0.2395 - precision_m: 0.2948 - recall_m: 0.2042 - val_loss: 1.3506 - val_acc: 0.2716 - val_f1_m: 0.0768 - val_precision_m: 0.2727 - val_recall_m: 0.0455
Epoch 3/50
30/30 [==============================] - 21s 716ms/step - loss: 1.3244 - acc: 0.3814 - f1_m: 0.2810 - precision_m: 0.4294 - recall_m: 0.2125 - val_loss: 1.3022 - val_acc: 0.2716 - val_f1_m: 0.1516 - val_precision_m: 0.3409 - val_recall_m: 0.1023
Epoch 4/50
30/30 [==============================] - 21s 690ms/step - loss: 1.2284 - acc: 0.4322 - f1_m: 0.3472 - precision_m: 0.5187 - recall_m: 0.2708 - val_loss: 1.2175 - val_acc: 0.3951 - val_f1_m: 0.3716 - val_precision_m: 0.85

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


CPU times: user 24min 38s, sys: 2min 52s, total: 27min 30s
Wall time: 19min 39s


In [33]:
xception_model_generator_2 = build_model_with_base(Xception, NUM_CLASES, IMG_SIZE[0], IMG_SIZE[1],0.00005,0)

hist_xception_generator_2 = xception_model_generator_2.fit(train_generator_2, validation_data=val_generator_2, epochs=50)


Epoch 1/50
30/30 [==============================] - 48s 828ms/step - loss: 1.6215 - acc: 0.2881 - f1_m: 0.1905 - precision_m: 0.2756 - recall_m: 0.1500 - val_loss: 1.3686 - val_acc: 0.2840 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/50
30/30 [==============================] - 24s 770ms/step - loss: 1.3608 - acc: 0.3941 - f1_m: 0.2584 - precision_m: 0.4067 - recall_m: 0.1917 - val_loss: 1.3416 - val_acc: 0.2963 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/50
30/30 [==============================] - 24s 781ms/step - loss: 1.1814 - acc: 0.4407 - f1_m: 0.3607 - precision_m: 0.5850 - recall_m: 0.2667 - val_loss: 1.2905 - val_acc: 0.3333 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 4/50
30/30 [==============================] - 24s 783ms/step - loss: 1.0548 - acc: 0.5297 - f1_m: 0.4000 - precision_m: 0.7300 - recall_m: 0.2875 - val_loss: 1.2423 - val_acc: 0.3333 - va

In [23]:
# Model training for Xception
%%time

hist_xception_preprocessed_2 = xception_model_preprocessed_2.fit(train_generator, validation_data=val_generator, epochs=50)
# Guardar modelos en formato H5
xception_model_preprocessed_2.save(os.path.join(save_dir, 'xception_model_preprocessed_2.h5'))


Epoch 1/50
30/30 [==============================] - 45s 737ms/step - loss: 2.1366 - acc: 0.2627 - f1_m: 0.2634 - precision_m: 0.3037 - recall_m: 0.2375 - val_loss: 1.4142 - val_acc: 0.2469 - val_f1_m: 0.2431 - val_precision_m: 0.3535 - val_recall_m: 0.1932
Epoch 2/50
30/30 [==============================] - 22s 720ms/step - loss: 1.1381 - acc: 0.5381 - f1_m: 0.4805 - precision_m: 0.6001 - recall_m: 0.4083 - val_loss: 1.2611 - val_acc: 0.3210 - val_f1_m: 0.2685 - val_precision_m: 0.4682 - val_recall_m: 0.1932
Epoch 3/50
30/30 [==============================] - 22s 712ms/step - loss: 0.8612 - acc: 0.6483 - f1_m: 0.6054 - precision_m: 0.7338 - recall_m: 0.5292 - val_loss: 1.1712 - val_acc: 0.4198 - val_f1_m: 0.4083 - val_precision_m: 0.5777 - val_recall_m: 0.3409
Epoch 4/50
30/30 [==============================] - 22s 708ms/step - loss: 0.8177 - acc: 0.6822 - f1_m: 0.6506 - precision_m: 0.7415 - recall_m: 0.5875 - val_loss: 1.2759 - val_acc: 0.4568 - val_f1_m: 0.3765 - val_precision_m: 0.

In [35]:
xception_model_generator_4 = build_model_with_base(Xception, NUM_CLASES, IMG_SIZE[0], IMG_SIZE[1],0.00005,0)

hist_xception_generator_4 = xception_model_generator_4.fit(train_generator_2, validation_data=val_generator_2, epochs=50)


Epoch 1/50
30/30 [==============================] - 50s 858ms/step - loss: 1.5833 - acc: 0.3305 - f1_m: 0.1768 - precision_m: 0.3044 - recall_m: 0.1292 - val_loss: 1.4079 - val_acc: 0.1728 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/50
30/30 [==============================] - 24s 774ms/step - loss: 1.2900 - acc: 0.4576 - f1_m: 0.3280 - precision_m: 0.5606 - recall_m: 0.2417 - val_loss: 1.3531 - val_acc: 0.2840 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/50
30/30 [==============================] - 24s 773ms/step - loss: 1.1113 - acc: 0.5636 - f1_m: 0.4278 - precision_m: 0.6693 - recall_m: 0.3292 - val_loss: 1.2863 - val_acc: 0.3580 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 4/50
30/30 [==============================] - 24s 791ms/step - loss: 1.0838 - acc: 0.5254 - f1_m: 0.4449 - precision_m: 0.6581 - recall_m: 0.3458 - val_loss: 1.2104 - val_acc: 0.4815 - va

# Evaluacion de modelos

# validacion de resultados

In [42]:
from tensorflow.keras.models import load_model
import os

save_dir = '/content/drive/My Drive/SavedModels'


xception_model_preprocessed.save(os.path.join(save_dir, 'final_xception_model_10.h5'))
xception_model_preprocessed_2.save(os.path.join(save_dir, 'final_xception_model_2.h5'))
xception_model_generator_2.save(os.path.join(save_dir, 'final_xception_model_v3.h5'))


In [25]:
# Evaluación en el conjunto de prueba
test_gen = ImageDataGenerator(rescale=1/255.)
test_generator = test_gen.flow_from_directory(directory=test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')



Found 83 images belonging to 4 classes.


In [26]:
import pandas as pd
from sklearn.metrics import classification_report
from tabulate import tabulate

def evaluate_and_print_metrics(model, model_name, test_generator):
    # Evaluación en el conjunto de prueba
    evaluation = model.evaluate(test_generator)

    # Obtener las predicciones para calcular otras métricas
    predictions = model.predict(test_generator)

    # Calcular métricas adicionales usando classification_report de scikit-learn
    report = classification_report(test_generator.classes, predictions.argmax(axis=1), target_names=test_generator.class_indices)

    # Obtener las métricas de precisión para cada modelo
    precision = evaluation[1]  # Extraer las métricas de interés
    f1 = float(report.split()[-2])
    recall = float(report.split()[-4])

    # Crear un DataFrame con las métricas
    df = pd.DataFrame({
        'Modelo': [model_name],
        'Precisión': [precision],
        'Recall': [recall],
        'F1': [f1]
        # Puedes agregar otras métricas si es necesario
    })

    # Muestra el DataFrame
    print(df)

    # Mostrar la tabla
    print(tabulate(df, headers='keys', tablefmt='pretty', showindex=False))

    return df



In [28]:
result_df = evaluate_and_print_metrics(xception_model_preprocessed, 'xception_model_v1', test_generator)


11/11 [==============================] - 3s 233ms/step
              Modelo  Precisión  Recall    F1
0  xception_model_v1   0.807229    0.27  0.27
+-------------------+--------------------+--------+------+
|      Modelo       |     Precisión      | Recall |  F1  |
+-------------------+--------------------+--------+------+
| xception_model_v1 | 0.8072289228439331 |  0.27  | 0.27 |
+-------------------+--------------------+--------+------+


In [29]:
result_df = evaluate_and_print_metrics(xception_model_preprocessed_2, 'xception_model_v2', test_generator)


11/11 [==============================] - 3s 219ms/step
              Modelo  Precisión  Recall    F1
0  xception_model_vv   0.819277    0.18  0.17
+-------------------+--------------------+--------+------+
|      Modelo       |     Precisión      | Recall |  F1  |
+-------------------+--------------------+--------+------+
| xception_model_vv | 0.8192771077156067 |  0.18  | 0.17 |
+-------------------+--------------------+--------+------+


In [39]:
result_df = evaluate_and_print_metrics(xception_model_generator_2, 'xception_model_v3', test_generator)


11/11 [==============================] - 3s 232ms/step
              Modelo  Precisión  Recall    F1
0  xception_model_v3    0.86747    0.26  0.26
+-------------------+--------------------+--------+------+
|      Modelo       |     Precisión      | Recall |  F1  |
+-------------------+--------------------+--------+------+
| xception_model_v3 | 0.8674699068069458 |  0.26  | 0.26 |
+-------------------+--------------------+--------+------+


In [40]:
result_df = evaluate_and_print_metrics(xception_model_generator_4, 'xception_model_v2', test_generator)


11/11 [==============================] - 4s 238ms/step
              Modelo  Precisión  Recall    F1
0  xception_model_v2   0.855422    0.22  0.22
+-------------------+--------------------+--------+------+
|      Modelo       |     Precisión      | Recall |  F1  |
+-------------------+--------------------+--------+------+
| xception_model_v2 | 0.8554216623306274 |  0.22  | 0.22 |
+-------------------+--------------------+--------+------+


In [ ]:
import matplotlib.pyplot as plt

def plot_hist(hist):
    # Crear la figura y los ejes
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plotear accuracy y val_accuracy
    ax.plot(hist.history["acc"], label="Train Accuracy", color="blue", linestyle="-")
    ax.plot(hist.history["val_acc"], label="Validation Accuracy", color="blue", linestyle="--")

    # Plotear precision_m y val_precision_m
    ax.plot(hist.history["precision_m"], label="Train Precision", color="green", linestyle="-")
    ax.plot(hist.history["val_precision_m"], label="Validation Precision", color="green", linestyle="--")

    # Configurar el subplot
    ax.set_title("Model Metrics")
    ax.set_ylabel("Metrics")
    ax.set_xlabel("Epoch")
    ax.legend(loc="upper left")

    # Mostrar la gráfica
    plt.show()

# Ejemplo de uso:
# Suponiendo que `hist` es el objeto devuelto por el entrenamiento del modelo
# model.fit(...)
# plot_hist(hist)


In [ ]:
import matplotlib.pyplot as plt

def plot_metrics_loss_f1(history, name):
    fig, ax = plt.subplots(figsize=(10, 6))
    fig.suptitle('Model Metrics Comparison', fontsize=16)

    # Pérdida
    ax.plot(history.history['loss'], label='Train Loss', color='blue', linestyle='-')
    ax.plot(history.history['val_loss'], label='Validation Loss', color='blue', linestyle='--')

    # F1 Score
    ax.plot(history.history['f1_m'], label='Train F1 Score', color='orange', linestyle='-')
    ax.plot(history.history['val_f1_m'], label='Validation F1 Score', color='orange', linestyle='--')

    ax.set_title(name)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Metrics')
    ax.legend()

    plt.show()

# Ejemplo de uso:
# Suponiendo que `history` y `name` son los datos proporcionados para un solo modelo
# plot_metrics_loss_f1(history, name)


In [ ]:
import matplotlib.pyplot as plt

def plot_hist(hist):
    # Crear la figura y los ejes
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plotear accuracy y val_accuracy
    ax.plot(hist.history["acc"], label="Train Accuracy", color="blue", linestyle="-")
    ax.plot(hist.history["val_acc"], label="Validation Accuracy", color="blue", linestyle="--")

    # Plotear precision_m y val_precision_m
    ax.plot(hist.history["precision_m"], label="Train Precision", color="green", linestyle="-")
    ax.plot(hist.history["val_precision_m"], label="Validation Precision", color="green", linestyle="--")

    # Configurar el subplot
    ax.set_title("Model Metrics")
    ax.set_ylabel("Metrics")
    ax.set_xlabel("Epoch")
    ax.legend(loc="upper left")

    # Mostrar la gráfica
    plt.show()

# Ejemplo de uso:
# Suponiendo que `hist` es el objeto devuelto por el entrenamiento del modelo
# model.fit(...)
# plot_hist(hist)


In [ ]:
modelos = ['Xception_10']


historias = [
   hist_xception_preprocessed
]
nombres = [
    'Xception (10 capas congeladas)',
]


In [ ]:
plot_hist(hist_xception_preprocessed)

In [ ]:
plot_metrics_loss_f1(hist_xception_preprocessed,'hist_xception_preprocessed')

In [ ]:
nombres